# Results Analysis

In [1]:
%matplotlib inline
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import numpy as np
import os
import time
from datetime import timedelta

from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn import neighbors
from sklearn import metrics
import regression_metrics as additional_metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score
#from neural_network import PyBrainNN

C:\Python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Python27\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
pd.options.display.float_format = '{:,.2f}'.format

### Load dataset

In [3]:
# read from csv
input_file_name = "results_id11010024.csv"
input_file_path = os.path.join(os.path.join(os.getcwd(), "data"), input_file_name)
test =  pd.read_csv(input_file_path, index_col=[0,1])

test.loc[20].tail()

testing_set_point = test.loc[20].shape[0]
print testing_set_point

27856


In [4]:
# read from csv
input_file_name = "id11010024_1_AR_WF_DT_dummy.csv"
input_file_path = os.path.join(os.path.join(os.getcwd(), "data", "online results", "LR"), input_file_name)
#res =  pd.read_csv(input_file_path, sep=';', names=['target', 'prediction'])
res =  pd.read_csv(input_file_path, sep=';', names=['target', 'prediction'])

res = res[-testing_set_point:]

print res.shape
res.head()

(27856, 2)


,target,prediction
77451,0.00,0.53
77452,0.00,0.32
77453,0.00,0.11
77454,1.00,0.30
77455,0.00,0.44


In [25]:
horizons = range(1,25)
#horizons = [1, 6, 12, 18, 24]
#horizons = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]

baselines = [{'name': "Last value", 'init': "NaiveModel('measurement')"},
             {'name': "Previous day", 'init': "NaiveModel('shifted_measurement_1d')"},
             {'name': "Previous week", 'init': "NaiveModel('shifted_measurement_1w')"},
             #{'name': "Measurement average", 'init': "NaiveModel('measurement_avr')"},
            ]

models = [{'name': "Ridge Regression", 'init': "linear_model.Ridge()", 'folder':'LR'},
          {'name': "K Nearest Neighbors", 'init': "neighbors.KNeighborsRegressor(n_neighbors = 5)", 'folder':'kNN'},
          {'name': "Random Forest", 'init': "RandomForestRegressor(n_estimators = 100, min_samples_leaf = 10, n_jobs=16)", 'folder':'RF100-5'},
          #{'name': "SVR", 'init': "svm.SVR(kernel ='rbf', C = 10, gamma = .001)"},
          #{'name': "Neural Network", 'init': "PyBrainNN(hidden_size = 100, learningrate = 0.001, momentum = 0.1, maxEpochs=10)"},
          ]

data_sets = [#{'name': "Baselines", 'ftrs': ar_ftrs},
             #{'name': "AR", 'ftrs': ar_ftrs},
             #{'name': "AR_WC_WF", 'ftrs': ar_wc_wf_ftrs},
             #{'name': "AR_DT", 'ftrs': ar_dt_ftrs},             
             #{'name': "AR_WC_DT", 'ftrs': ar_wc_dt_ftrs},
             {'name': "AR_WF_DT"},    
             #{'name': "ALL", 'ftrs': all_ftrs},
             #{'name': "TOP_20", 'ftrs': top20_ftrs},
             #{'name': "TOP_10", 'ftrs': top10_ftrs},
             ]

error_metrics = [{'name': "R2 Score", 'short': "R2", 'function': metrics.r2_score},
                 #{'name': "Explained Variance Score", 'short': "EV", 'function': metrics.explained_variance_score},
                 {'name': "Mean Absolute Error", 'short': "MAE", 'function': metrics.mean_absolute_error},
                 #{'name': "Mean Squared Error", 'short': "MSE", 'function': metrics.mean_squared_error},
                 {'name': "Mean Absolute Percentage Error", 'short': "MAPE", 'function': additional_metrics.mean_absolute_percentage_error},
                 #{'name': "Mean Absolute Scaled Error", 'short': "MASE", 'function': additional_metrics.mean_absolute_scaled_error}
                ]

#validation_df = pd.DataFrame(index=pd.MultiIndex.from_arrays([[],[],[]], names=['sensor', 'horizon', 'error metric']),
#                             columns=pd.MultiIndex.from_arrays([[],[]], names=['method', 'data set']))

In [26]:
validation_df = pd.DataFrame(index=pd.MultiIndex.from_arrays([[],[],[]], names=['horizon', 'dataset', 'model']),
                             columns=pd.MultiIndex.from_arrays([[],[]], names=['sensor', 'results']))

sensor_list = ["id12041992", "id11010024", "id14061721", "id12041022"]
#sensor_list = ['id12041022']
#horizons = range(1,25)
#data_sets = ["AR_WF_DT"]

for sensor_name in sensor_list:   
    for horizon in horizons:
        for data_set in data_sets:
            for model in models: 

                # construct file path
                if model['name'] == "Ridge Regression":
                    input_file_name = "{}_{}_{}_dummy.csv".format(sensor_name, horizon, data_set['name'])
                else:
                    input_file_name = "{}_{}_{}.csv".format(sensor_name, horizon, data_set['name'])

                input_file_path = os.path.join(os.path.join(os.getcwd(), "data", "online results", model['folder']), input_file_name)

                # read file
                if model['name'] == "Ridge Regression":
                    res_df =  pd.read_csv(input_file_path, sep=';', names=['target', 'prediction'])
                else: 
                    res_df =  pd.read_csv(input_file_path, sep=';')
                    
                # take only last n values
                #testing_set_point TODO
                res_df[-testing_set_point:]
                
                # compute error metric
                for metric in error_metrics:
                    # TODO
                    true = res_df['target'].values
                    pred = res_df['prediction'].values
                    
                    error = metric['function'](true, pred)
                    
                    validation_df.loc[(horizon, data_set['name'], model['name']), (sensor_name, metric['short'])] = error

validation_df

sensor                               id12041992             id11010024       \
results                                      R2   MAE  MAPE         R2  MAE   
horizon dataset  model                                                        
1.00    AR_WF_DT Ridge Regression          0.80 11.78 12.70       0.71 0.62   
                 K Nearest Neighbors       0.65 15.64 16.45       0.60 0.65   
                 Random Forest             0.81 11.87 12.96       0.77 0.53   
2.00    AR_WF_DT Ridge Regression          0.75 13.42 14.64       0.65 0.68   
                 K Nearest Neighbors       0.46 19.44 20.54       0.45 0.76   
                 Random Forest             0.76 13.22 14.47       0.72 0.58   
3.00    AR_WF_DT Ridge Regression          0.73 13.93 15.27       0.63 0.71   
                 K Nearest Neighbors       0.34 21.28 22.69       0.34 0.84   
                 Random Forest             0.75 13.74 15.12       0.71 0.60   
4.00    AR_WF_DT Ridge Regression          0.72 14.09 15.50       0.61 0.72   
                 K Nearest Neighbors       0.25 22.41 24.14       0.22 0.90   
                 Random Forest             0.74 14.00 15.47       0.69 0.61   
5.00    AR_WF_DT Ridge Regression          0.72 14.16 15.61       0.59 0.73   
                 K Nearest Neighbors       0.21 23.03 24.96       0.17 0.92   
                 Random Forest             0.73 14.13 15.61       0.68 0.62   
6.00    AR_WF_DT Ridge Regression          0.71 14.26 15.74       0.59 0.74   
                 K Nearest Neighbors       0.20 23.15 25.07       0.16 0.92   
                 Random Forest             0.72 14.36 15.94       0.67 0.63   
7.00    AR_WF_DT Ridge Regression          0.71 14.35 15.86       0.58 0.74   
                 K Nearest Neighbors       0.22 22.89 24.58       0.17 0.91   
                 Random Forest             0.72 14.46 16.06       0.66 0.64   
8.00    AR_WF_DT Ridge Regression          0.71 14.35 15.89       0.58 0.74   
                 K Nearest Neighbors       0.26 22.09 23.51       0.18 0.91   
                 Random Forest             0.72 14.38 15.98       0.66 0.64   
9.00    AR_WF_DT Ridge Regression          0.71 14.31 15.85       0.58 0.74   
                 K Nearest Neighbors       0.30 21.31 22.46       0.22 0.88   
                 Random Forest             0.72 14.37 15.98       0.65 0.64   
10.00   AR_WF_DT Ridge Regression          0.71 14.32 15.83       0.58 0.74   
                 K Nearest Neighbors       0.32 21.13 22.22       0.26 0.86   
                 Random Forest             0.72 14.37 15.89       0.65 0.65   
...                                         ...   ...   ...        ...  ...   
15.00   AR_WF_DT Ridge Regression          0.71 14.41 15.99       0.58 0.74   
                 K Nearest Neighbors       0.36 20.92 22.97       0.33 0.83   
                 Random Forest             0.71 14.50 16.03       0.64 0.64   
16.00   AR_WF_DT Ridge Regression          0.71 14.40 15.97       0.58 0.74   
                 K Nearest Neighbors       0.40 20.34 22.51       0.32 0.83   
                 Random Forest             0.72 14.44 15.91       0.65 0.64   
17.00   AR_WF_DT Ridge Regression          0.71 14.38 15.93       0.58 0.75   
                 K Nearest Neighbors       0.43 19.81 21.98       0.32 0.83   
                 Random Forest             0.72 14.46 15.97       0.64 0.64   
18.00   AR_WF_DT Ridge Regression          0.71 14.35 15.89       0.58 0.75   
                 K Nearest Neighbors       0.47 19.26 21.37       0.32 0.83   
                 Random Forest             0.72 14.39 15.85       0.65 0.64   
19.00   AR_WF_DT Ridge Regression          0.71 14.32 15.86       0.57 0.75   
                 K Nearest Neighbors       0.50 18.84 20.86       0.32 0.83   
                 Random Forest             0.72 14.37 15.86       0.66 0.64   
20.00   AR_WF_DT Ridge Regression          0.71 14.31 15.83       0.57 0.75   
                 K Nearest Neighbors       0.51 18

## Evaluation Scores Over All Horizons

MF10-10

In [10]:
validation_df.groupby(level=['dataset', 'model'], sort=False).mean()

sensor                       id12041992             id11010024             \
results                              R2   MAE  MAPE         R2  MAE  MAPE   
dataset  model                                                              
AR_WF_DT Ridge Regression          0.72 14.18 15.66       0.59 0.73 44.30   
         K Nearest Neighbors       0.39 20.20 21.81       0.30 0.84 54.31   
         Random Forest             0.71 14.91 17.12       0.64 0.70 42.92   

sensor                       id14061721            id12041022               
results                              R2  MAE  MAPE         R2    MAE  MAPE  
dataset  model                                                              
AR_WF_DT Ridge Regression          0.76 1.79 30.62       0.85  78.89 19.34  
         K Nearest Neighbors       0.56 2.13 30.77       0.73 100.19 24.36  
         Random Forest             0.77 1.74 28.97       0.90  67.72 16.68

RF100-5

In [27]:
validation_df.groupby(level=['dataset', 'model'], sort=False).mean()

sensor                       id12041992             id11010024             \
results                              R2   MAE  MAPE         R2  MAE  MAPE   
dataset  model                                                              
AR_WF_DT Ridge Regression          0.72 14.18 15.66       0.59 0.73 44.30   
         K Nearest Neighbors       0.39 20.20 21.81       0.30 0.84 54.31   
         Random Forest             0.73 14.17 15.64       0.66 0.63 36.37   

sensor                       id14061721            id12041022               
results                              R2  MAE  MAPE         R2    MAE  MAPE  
dataset  model                                                              
AR_WF_DT Ridge Regression          0.76 1.79 30.62       0.85  78.89 19.34  
         K Nearest Neighbors       0.56 2.13 30.77       0.73 100.19 24.36  
         Random Forest             0.77 1.62 24.46       0.88  71.45 15.83

RF30-5

In [22]:
validation_df.groupby(level=['dataset', 'model'], sort=False).mean()

sensor                       id12041992             id11010024             \
results                              R2   MAE  MAPE         R2  MAE  MAPE   
dataset  model                                                              
AR_WF_DT Ridge Regression          0.72 14.18 15.66       0.59 0.73 44.30   
         K Nearest Neighbors       0.39 20.20 21.81       0.30 0.84 54.31   
         Random Forest             0.72 14.24 15.71       0.65 0.63 36.80   

sensor                       id14061721            id12041022               
results                              R2  MAE  MAPE         R2    MAE  MAPE  
dataset  model                                                              
AR_WF_DT Ridge Regression          0.76 1.79 30.62       0.85  78.89 19.34  
         K Nearest Neighbors       0.56 2.13 30.77       0.73 100.19 24.36  
         Random Forest             0.76 1.63 24.58       0.88  72.36 16.08

### Save to file

In [28]:
output_file_name = "online_validationResults_RF100_5.csv"
output_file_path = os.path.join(os.path.join(os.getcwd(), "data"), output_file_name)

validation_df.to_csv(output_file_path)

### Test read file

In [29]:
# read from csv
test =  pd.read_csv(output_file_path, index_col=[0,1,2], header=[0,1])

test.groupby(level=['dataset', 'model'], sort=False).mean()

sensor                       id12041992             id11010024             \
results                              R2   MAE  MAPE         R2  MAE  MAPE   
dataset  model                                                              
AR_WF_DT Ridge Regression          0.72 14.18 15.66       0.59 0.73 44.30   
         K Nearest Neighbors       0.39 20.20 21.81       0.30 0.84 54.31   
         Random Forest             0.73 14.17 15.64       0.66 0.63 36.37   

sensor                       id14061721            id12041022               
results                              R2  MAE  MAPE         R2    MAE  MAPE  
dataset  model                                                              
AR_WF_DT Ridge Regression          0.76 1.79 30.62       0.85  78.89 19.34  
         K Nearest Neighbors       0.56 2.13 30.77       0.73 100.19 24.36  
         Random Forest             0.77 1.62 24.46       0.88  71.45 15.83